1. 只要环境变量中有 HADOOP_CLASSPATH , 就可以启动 Flink on yarn
```shell
# 示例
[@10.18.17.54 /opt/scripts/flink/streaming]$ echo $HADOOP_HOME
/opt/hadoop/hadoop
[@10.18.17.54 /opt/scripts/flink/streaming]$ ls /opt/hadoop/hadoop
bin  etc  hs_err_pid550.log  include  lib  libexec  LICENSE.txt  NOTICE.txt  README.txt  sbin  share  TestDFSIO_results.log
```

2. 推荐使用 Application Mode, 可以更好地资源隔离. (相比 session mode)    
  Application Mode会启动一个 Flink 集群在 YARN 上, 集群启动后, 可以用命令行交互  
```shell
# List running job on the cluster
./bin/flink list -t yarn-application -Dyarn.application.id=application_XXXX_YY
# Cancel running job
./bin/flink cancel -t yarn-application -Dyarn.application.id=application_XXXX_YY <jobId>
```

3. `flink --help` 查看启动参数的 shortcut

4. 关于 yarn jobmanager 的 HA
[zk HA配置](https://nightlies.apache.org/flink/flink-docs-release-1.16/docs/deployment/ha/zookeeper_ha/). 不配 HA 的话, yarn 更在意 jobmanager 的失败重启策略. 最大重启次数收2个影响  
```shell
yarn.application-attempts=2 # ApplicationMaster 的重启次数, 会让整个 flink 集群重启, client 连接也会断开
                   # 这个值受 yarn 的 yarn.resourcemanager.am.max-attempts=2 配置限制
```

5. 启动命令示例

```shell
nohup \
flink run \
-d \   # yarn application detach mode
-p 20 \ # --parallelism: 执行任务的并行度
-m yarn-cluster \ # jobManager的连接地址, 可以覆盖默认配置
-yjm 1G \       # jobManager 内存大小
-ytm 4G \       # taskManager container 大小
-yn 12 \       # 开启几个 yarncontainer, 也就是 taskManager 的数量. 因为 flink 集群的 jobmanager 是单点的, 即使配置了 zk HA
-ys 2 \        # 每个 taskmanager 的 slots 数量
-ynm "APP_STATISTICS" \ # yarn 应用的名字
-yqu root.yarn \     # yarn 应用的队列名
-yD env.java.opts.taskmanager="-Djasypt.encryptor.password=tPpGsLBEIcVwx4y2" \  # -yD <property=value> 指定参数名
-c com.sohu.tv.stream.overview.app.APPStatistics \  # main class
$dir/flink-stream-1.0-SNAPSHOT.jar
```

6. slot数量和并行度大小   
 二者应该设置的一样大. 至少 slot数量 > 并行度, 否则集群启动不来   
 [flink-architecture](https://nightlies.apache.org/flink/flink-docs-release-1.16/docs/concepts/flink-architecture/)   
 [job_scheduling](https://nightlies.apache.org/flink/flink-docs-release-1.16/docs/internals/job_scheduling/)